In [6]:
##################################### IMPORTS #######################################

import pygame
import numpy as np
from random import randrange
import random

#####################################################################################

############################### FONCTIONS DE BASES ##################################

def grille():
    """ Calcule et dessine la nouvelle grille """
    protocole(),draw()
    pygame.display.flip()  # Rafraîchit l'affichage

def initialisation(dimension):
    """ Initialisation | Renvoie la hauteur, la largeur et les matrices des etats """
    hauteur, largeur = dimension,dimension
    
    # Matrice qui gèrera l'aspect graphique des cellules (rectangles)
    cellule = [[None for _ in range(hauteur)] for _ in range(largeur)]
    # Matrice qui contient l'état actuel de chaque cellules [ 0=morte / 1=vivante], elles sont toutes mortes au début
    etat = [[0 for _ in range(hauteur)] for _ in range(largeur)]
    # Matrice qui contient le prochain état de chaque cellule (la matrice précedente prendra sa valeur)
    etat_futur = [[0 for _ in range(hauteur)] for _ in range(largeur)]
    
    for y in range(hauteur):
        for x in range(largeur):
            etat[x][y] = 0
            etat_futur[x][y] = 0
            # Création des rectangles
            cellule[x][y] = pygame.Rect(x*taille_cellule, y*taille_cellule, taille_cellule, taille_cellule) 
            
    # Place environ 25% de cellules vivantes
    for i in range(largeur * hauteur // 4):
        etat[randrange(largeur)][randrange(hauteur)] = 1
        
    return hauteur, largeur, cellule, etat, etat_futur

def initialisation_aleatoire():
    """ Initialisation aléatoire de la grille : intervalle régulable """
    dimension = random.randint(50, 300)
    hauteur, largeur, cellule, etat, etat_futur = initialisation(x)
    return dimension, hauteur, largeur, cellule, etat, etat_futur

def protocole():
    """ Calcule et applique les règles """
    for y in range(hauteur):
        for x in range(largeur):
            nombre_voisins = compte_voisins_vivant(x, y)
            # Applique les règles de survie et de naissance des cellules
            if etat[x][y] == 1 and nombre_voisins < 2:
                etat_futur[x][y] = 0
            elif etat[x][y] == 1 and nombre_voisins > 3:
                etat_futur[x][y] = 0
            elif etat[x][y] == 1 and (nombre_voisins == 2 or nombre_voisins == 3):
                etat_futur[x][y] = 1
            elif etat[x][y] == 0 and nombre_voisins == 3:
                etat_futur[x][y] = 1
        
    for y in range(hauteur):
        for x in range(largeur):
            etat[x][y] = etat_futur[x][y]

def compte_voisins_vivant(x, y):
    """ Calcul et renvoie le nombre de cellules voisines en vie (tableau torique) """
    nb_voisins = 0
    # On definit une matrice contenant les coordonnées de tous les voisins d'une cellule x=0 et y=0
    directions = [(-1, 1), (0, 1), (1, 1), (-1, 0), (1, 0), (-1, -1), (0, -1), (1, -1)]
    
    for dx, dy in directions:
        # on utilise le % pour que le bord gauche soit connecté au bord droit, 
        # et le bord supérieur soit connecté au bord inférieur
        if etat[(x + dx) % largeur][(y + dy) % hauteur] == 1:
            nb_voisins += 1
    return nb_voisins

def draw():
    """ Dessine / Redessine toutes les cellules """
    fenetre.fill((255, 255, 255))  # Fond
    
    for y in range(hauteur):
        for x in range(largeur):
            couleur = (0, 0, 0) if etat[x][y] == 1 else (255, 255, 255)
            pygame.draw.rect(fenetre, couleur, cellule[x][y])
    
    # Dessiner le cadre noir à droite
    cadre_largeur = largeur_panel_droit  # Largeur du cadre
    cadre_rect = pygame.Rect(taille_cellule * largeur, 0, cadre_largeur, taille_cellule * hauteur)
    # Dessiner le rectangle blanc à l'intérieur du cadre
    centre_rect = pygame.Rect(taille_cellule * largeur + 10, 10, cadre_largeur - 20, taille_cellule * hauteur - 20)
    pygame.draw.rect(fenetre, (232, 220, 202), centre_rect)  # Dessine le centre blanc
    # Dessiner le contour noir
    pygame.draw.rect(fenetre, (0, 0, 0), cadre_rect, 10)  # Le dernier argument (2) définit l'épaisseur du contour

#####################################################################################

############################### FONCTIONS SUPLEMENTAIRES ############################

def modifie_etat_cellule(x, y):
    """ Modifie l'état de la cellule aux coordonnées x,y (largeur, hauteur) à chaque clic de souris """
    if x <= largeur * taille_cellule and y <= hauteur * taille_cellule:
        # Calcule les indices de la cellule cliquée
        colonne = x // taille_cellule
        ligne = y // taille_cellule
        
        # Inverse l'état de la cellule
        etat[colonne][ligne] = 1 if etat[colonne][ligne] == 0 else 0
        
        print(f"Cellule à ({colonne}, {ligne}) maintenant {'vivante' if etat[colonne][ligne] == 1 else 'morte'}")
    

#####################################################################################

##################################### LANCEMENT #####################################

# Initialisation de pygame et de la fenêtre
pygame.init()

# Initialisation du jeu
taille_cellule = 7
dimension = 100
hauteur, largeur, cellule, etat, etat_futur = initialisation(dimension)
#dimension, hauteur, largeur, cellule, etat, etat_futur = initialisation_aleatoire() # pour initialiser aléatoirement la grille

# Creation fenetre : elle sera de largeur : taille_cellule*largeur et hauteur : taille_cellule*hauteur  
largeur_panel_droit = 400
fenetre = pygame.display.set_mode((taille_cellule*largeur + largeur_panel_droit, taille_cellule*hauteur))
pygame.display.set_caption("Projet : Jeu de la vie")  
    
# Boucle de jeu
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:  # Vérifie si un clic de souris a eu lieu
            pos_x, pos_y = pygame.mouse.get_pos()  # Récupère les coordonnées de la souris
            modifie_etat_cellule(pos_x, pos_y)
            

    # Mise à jour et affichage de la grille
    grille()
    pygame.time.delay(20)  # Vitesse de l'animation (combien de miliseconde entre chaque changement d'état)

# Fermeture de pygame
pygame.quit()

#####################################################################################

Cellule à (61, 49) maintenant morte
Cellule à (24, 48) maintenant vivante
Cellule à (21, 37) maintenant vivante
Cellule à (25, 59) maintenant vivante
Cellule à (56, 55) maintenant vivante
Cellule à (57, 39) maintenant vivante
Cellule à (95, 45) maintenant vivante
Cellule à (99, 53) maintenant vivante
